# Visualise results from model trained in clustering_analysis_training.ipynb

Author: Katharine Leney, April 2025

In [8]:
import joblib
import pandas as pd
import pickle
import os
import sys
import plotly.express as px  
sys.path.append("../src")
from utils.filter_dataframe import filter_dataframe_by_year, get_top_topics
from utils.parse_model_folder import parse_model_folder
from utils.generate_clean_labels import generate_clean_labels
from utils.generate_opportunity_table import generate_opportunity_table

In [9]:
# ====================================
# Load model and associated data
# ====================================

# Model name
model_name = f"bertopic_cluster_model"

# Build correct path
model_path = os.path.join("../models/", model_name, "model")

# Load the model
topic_model = joblib.load(model_path)

# Load texts and timestamps
with open(os.path.join("../models/", model_name, "texts.pkl"), "rb") as f:
    texts = pickle.load(f)

with open(os.path.join("../models/", model_name, "timestamps.pkl"), "rb") as f:
    timestamps = pickle.load(f)

model_label = parse_model_folder(model_name)
print(model_label)

# Make sure the outputs folder exists
os.makedirs("../outputs", exist_ok=True)

In [10]:
# Explore top topics
topic_model.get_topic_info().head(10)
#topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,595,-1_fuel_airlines_airline_emissions,"[fuel, airlines, airline, emissions, passenger...",[Region 2023 2022 5-year average (2019-2023) A...
1,0,233,0_safety_iosa_accidents_accident,"[safety, iosa, accidents, accident, audit, rat...",[22 Auditing to global standards and practices...
2,1,194,1_security_information_passenger_smart security,"[security, information, passenger, smart secur...",[Smart Security Smart Security is joint initia...
3,2,170,2_settlement_billion_processed_bsp,"[settlement, billion, processed, bsp, cass, is...",[60 09 PASSENGER EXPERIENCE FINANCIAL SYSTEMS ...
4,3,146,3_charges_airport_airports_infrastructure,"[charges, airport, airports, infrastructure, c...",[Airports Savings in airport charges of $1.5 b...
5,4,143,4_business_global_governments_important,"[business, global, governments, important, gre...",[Tony Tyler Director General & CEO Internation...
6,5,92,5_ndc_order_travel_airline,"[ndc, order, travel, airline, retailing, distr...",[As IATA receives feedback from early NDC depl...
7,6,90,6_baggage_fast travel_fast_boarding,"[baggage, fast travel, fast, boarding, passeng...",[The program provides self-service options at ...
8,7,89,7_cargo_e-freight_freight_e-awb,"[cargo, e-freight, freight, e-awb, chain, supp...","[In partnership with shippers, freight forward..."
9,8,84,8_saf_corsia_emissions_production,"[saf, corsia, emissions, production, carbon, i...",[Sustainable aviation fuels One of the most pr...


In [11]:
# ------------------------------------------------------------
# Visualize intertopic distances
# (move this to a function)
# ------------------------------------------------------------

# N.B. UMAP (Uniform Manifold Approximation and Projection)
# used for dimensionality reduction. 
fig_2d = topic_model.visualize_topics(custom_labels=True)

# Update background and grid
fig_2d.update_layout(
    plot_bgcolor="white",
    paper_bgcolor="white",
    font_family="Arial",
    title={
        "text": f"Intertopic Distance Map<br><sup>{model_label}</sup>",
        "x": 0.15,
        "font": {"size": 20}
    },
    width=1000,
    height=800,
    xaxis=dict(
        showgrid=False,
        zeroline=True,
        showline=True,
        ticks="outside",
        title=None
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=True,
        showline=True,
        ticks="outside",
        title=None
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    )
)
fig_2d.write_html("../outputs/intertopic_distance_map_2D.html")   

fig_2d.show()

In [12]:
# ------------------------------------------------------
# Clean up labels and remove junk topics
# ------------------------------------------------------

# Retrieve the current model labels before overwriting
old_labels = topic_model.topic_labels_

# Generate new labels
clean_labels, junk_topics = generate_clean_labels(topic_model, top_n_words=1)

# Print label changes for review
print("\nGenerated topic labels:")
for topic_id, label in sorted(clean_labels.items()):
    old_label = old_labels.get(topic_id)
    print(f"Topic {topic_id}: {label} (OLD: {old_label})")

# Apply to the model
topic_model.set_topic_labels(clean_labels)

Topic  23 identified as junk:
[('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05)]

Generated topic labels:
Topic -1: NULL (OLD: -1_fuel_airlines_airline_emissions)
Topic 0: Safety (OLD: 0_safety_iosa_accidents_accident)
Topic 1: Security (OLD: 1_security_information_passenger_smart security)
Topic 2: Settlement (OLD: 2_settlement_billion_processed_bsp)
Topic 3: Charges (OLD: 3_charges_airport_airports_infrastructure)
Topic 4: Business (OLD: 4_business_global_governments_important)
Topic 5: New Distribution Capability (OLD: 5_ndc_order_travel_airline)
Topic 6: Baggage (OLD: 6_baggage_fast travel_fast_boarding)
Topic 7: Cargo (OLD: 7_cargo_e-freight_freight_e-awb)
Topic 8: SAF (OLD: 8_saf_corsia_emissions_production)
Topic 9: Regulation (OLD: 9_regulation_rights_passenger rights_regulations)
Topic 10: Markets (OLD: 10_markets_growth_load_premium)
Topic 11: Covid-19 (OLD: 11_covid-19_health_testing_travel)
Topi

In [13]:
# ------------------------------------------------------------
# Generate topic time series and visualize trends
# ------------------------------------------------------------

timestamps = pd.to_datetime(timestamps, format="%Y")
topics_over_time = topic_model.topics_over_time(texts, timestamps, topic_model.topics_)

# OPTIONAL: Set filter criteria
# Set to None to ignore
min_year = 2015
max_year = None
exclude_years = None #[2020]  

# Apply year filtering
filtered_topics, filter_label = filter_dataframe_by_year(
    topics_over_time,
    min_year=min_year,
    max_year=max_year,
    exclude_years=exclude_years
)

# Apply topic filtering (remove junk and noise topics)
filtered_topics = filtered_topics[
    (~filtered_topics["Topic"].isin(junk_topics)) &
    (filtered_topics["Topic"] != -1)
]

# Select top k most frequent clean topics
top_k = 8
top_topics = get_top_topics(filtered_topics, top_k)

# Make plot of topic frequency by year
# y-axis = % of all text chunks assigned to this topic in a given year
fig = topic_model.visualize_topics_over_time(filtered_topics, topics=top_topics)

# Apply custom colors and labels
palette = px.colors.qualitative.T10

for i, trace in enumerate(fig.data):
    topic_id = top_topics[i]
    trace.line.color    = palette[i % len(palette)]
    trace.name = clean_labels[topic_id]
    hover_text = trace.hovertemplate or ""
    trace.hovertemplate = f"{clean_labels[topic_id]}<extra></extra>"

# Clean up the figure layout
fig.update_layout(
    title=f"Topic Trends Over Time<br><sup>{filter_label}</sup>",
    width=1200,
    height=600,
    xaxis_title="Year",
    yaxis_title="Topic Frequency (% of Annual Report)",
    legend_title="Topics",
    font=dict(family="Arial", size=12),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.4,
        xanchor="center",
        x=0.5
    )
)

output_filename = f"../outputs/topic_trends"
fig.write_html(f"{output_filename}.html")
fig.write_image(f"{output_filename}.png")

fig.show()

19it [00:05,  3.31it/s]


In [14]:
# ===============================================
# Generate a table matching trends to business
# opportunities, and assign a priority ranking
# based on trend size and growth
# ===============================================

matched_opportunities_df, unmatched_topics = generate_opportunity_table(topic_model, topics_over_time)

print(f"Filtering (if any): {model_label}")
print(f"Model Name: {model_name}\n")
print("=== Matched Business Opportunities ===\n")
display(matched_opportunities_df)

# Save table as CSV
output_filename = f"../outputs/business_opportunities.csv"
matched_opportunities_df.to_csv(output_filename, index=False)

print(f"\nSaved opportunity table to: {output_filename}")

Filtering (if any): 
Model Name: bertopic_cluster_model

=== Matched Business Opportunities ===



,Trend,Detected Label,Implication,Opportunity,Priority (Recommended),Keyword Fraction (%),Topic Growth
0,Safety,0_safety_iosa_accidents_accident,Continued industry focus on safety standards,Invest in IOSA certification and safety audits,High,9.62,7
1,Airport,3_charges_airport_airports_infrastructure,Growing airport-related fees and costs,Negotiate airport charges and improve operatio...,High,6.03,6
2,Security,1_security_information_passenger_smart security,Passenger data security and travel security co...,Enhance digital identity management and passen...,Medium,8.01,4
3,Settlement,2_settlement_billion_processed_bsp,"Changes in financial settlement systems (BSP, ...",Optimize settlement systems and implement fast...,Medium,7.02,-1
4,Business,4_business_global_governments_important,Organizational agility and change management,Simplify business processes and accelerate cha...,Medium,5.91,-7
5,Cargo,7_cargo_e-freight_freight_e-awb,Continued strength in air cargo and freight de...,Expand cargo services and invest in logistics ...,Medium,3.68,2
6,Emissions,8_saf_corsia_emissions_production,Increasing pressure for environmental responsi...,Adopt Sustainable Aviation Fuels (SAF) and car...,Medium,3.47,15
7,Growth,10_markets_growth_load_premium,Strong recovery in air travel post-pandemic,Expand network capacity and optimize route pla...,Medium,2.93,1
8,Tax,13_tax_taxes_taxation_government,Government pressure via taxes on aviation,Engage in advocacy and optimize tax planning s...,Medium,2.73,2
9,Growth,15_cargo_trade_growth_global,Strong recovery in air travel post-pandemic,Expand network capacity and optimize route pla...,Medium,2.48,5



Saved opportunity table to: ../outputs/business_opportunities.csv
